# Bayesian prediction of Brownian particle, as in the [Nature paper](nature.com/articles/s41567-020-1018-2)

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch

In [4]:
from brownian_datagen import BrownianDatagen
from bayesian import posterior_predictive_forward_and_backward, ExperimentResults, BayesTrainData

In [5]:
brownian = BrownianDatagen()
save_dir = "20230724_preds/brownian00/"

In [6]:
meaningless_time_series = np.zeros((5, 1), np.float32)
dt = BayesTrainData(meaningless_time_series)
# For Brownian motion, different windows might come from different
# trajectories, so having one single time series doesn't make sense.
# Morover, backward and forward don't refer to the same thing anymore:
# they are generated separately, rather than backward = np.flip(forward).
# But I want to keep using the same class for holding the train data, so I set attributes manually.
dt.windows_f, dt.targets_f = brownian.windows_targets(window_len=3, numParticles=50)
dt.windows_b, dt.targets_b = brownian.windows_targets(window_len=3, backward=True, numParticles=50)
dt.windows_f, dt.targets_f, dt.windows_b, dt.targets_b = map(
    lambda x: torch.tensor(x, dtype=torch.float32),
    (dt.windows_f, dt.targets_f, dt.windows_b, dt.targets_b))

In [12]:
posterior_predictive_forward_and_backward(dt, save_dir=save_dir, num_samples=10)

Sample: 100%|████████████████████████████████████████████| 20/20 [00:01, 19.60it/s, step size=9.91e-04, acc. prob=0.702]                           


(Predictive(
   (model): BayesianThreeFCLayers(
     (fc1): PyroLinear(in_features=3, out_features=10, bias=True)
     (relu1): ReLU()
     (fc2): PyroLinear(in_features=10, out_features=10, bias=True)
     (relu2): ReLU()
     (fc3): PyroLinear(in_features=10, out_features=1, bias=True)
   )
 ),
 Predictive(
   (model): BayesianThreeFCLayers(
     (fc1): PyroLinear(in_features=3, out_features=10, bias=True)
     (relu1): ReLU()
     (fc2): PyroLinear(in_features=10, out_features=10, bias=True)
     (relu2): ReLU()
     (fc3): PyroLinear(in_features=10, out_features=1, bias=True)
   )
 ))

In [7]:
!ls "20230724_preds/brownian00/"

noisy_ts.torch		   targets_b.torch  windows_b.torch
predictive.backward.torch  targets_f.torch  windows_f.torch
predictive.forward.torch   ts.torch


In [15]:
er = ExperimentResults(save_dir)
((er.pred_obs_b.mean(axis=0) - er.targets_b) ** 2).mean().sqrt()

tensor(0.2283)